# Analyze Small N Solutions

Check if our solutions for small N values match known optimal packings.

In [ ]:
import pandas as pd
import numpy as np
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import matplotlib.pyplot as plt

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_tree_polygon(cx, cy, angle_deg):
    angle_rad = np.radians(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    x = TX * cos_a - TY * sin_a + cx
    y = TX * sin_a + TY * cos_a + cy
    return Polygon(zip(x, y))

def get_bounding_box_side(trees):
    all_poly = unary_union(trees)
    bounds = all_poly.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

# Load current solution
df = pd.read_csv('/home/submission/submission.csv')
df['x'] = df['x'].str.strip('s').astype(float)
df['y'] = df['y'].str.strip('s').astype(float)
df['deg'] = df['deg'].str.strip('s').astype(float)
df['n'] = df['id'].str[:3].astype(int)

print("Current solutions for N=1-10:")
print("="*60)
for n in range(1, 11):
    group = df[df['n'] == n]
    trees = [get_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    score = side**2 / n
    print(f"N={n}: side={side:.6f}, score={score:.6f}")
    for _, row in group.iterrows():
        print(f"  Tree: x={row['x']:.4f}, y={row['y']:.4f}, deg={row['deg']:.2f}")

In [ ]:
# Visualize N=2 solution
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for idx, n in enumerate(range(1, 11)):
    ax = axes[idx // 5, idx % 5]
    group = df[df['n'] == n]
    
    for _, row in group.iterrows():
        tree = get_tree_polygon(row['x'], row['y'], row['deg'])
        x, y = tree.exterior.xy
        ax.fill(x, y, alpha=0.5)
        ax.plot(x, y, 'b-', linewidth=0.5)
    
    trees = [get_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    
    ax.set_aspect('equal')
    ax.set_title(f'N={n}, side={side:.4f}')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/home/code/experiments/005_random_restart/small_n_visualization.png', dpi=100)
plt.show()
print("Saved visualization to small_n_visualization.png")

In [ ]:
# Calculate theoretical lower bounds
# For N=1, the optimal is a single tree at 45 degrees
# The bounding box of a tree at 45 degrees has side = sqrt(2) * max_dimension / 2

print("\nTheoretical analysis:")
print("="*60)

# Single tree at 45 degrees
tree_45 = get_tree_polygon(0, 0, 45)
bounds = tree_45.bounds
side_45 = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
print(f"Single tree at 45 degrees: side = {side_45:.6f}")

# Single tree at 0 degrees
tree_0 = get_tree_polygon(0, 0, 0)
bounds = tree_0.bounds
side_0 = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
print(f"Single tree at 0 degrees: side = {side_0:.6f}")

# The optimal angle for N=1 minimizes the bounding box
print("\nSearching for optimal angle for N=1...")
best_side = float('inf')
best_angle = 0
for angle in range(0, 360):
    tree = get_tree_polygon(0, 0, angle)
    bounds = tree.bounds
    side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
    if side < best_side:
        best_side = side
        best_angle = angle

print(f"Optimal angle for N=1: {best_angle} degrees, side = {best_side:.6f}")
print(f"Current N=1 side: {get_bounding_box_side([get_tree_polygon(df[df['n']==1].iloc[0]['x'], df[df['n']==1].iloc[0]['y'], df[df['n']==1].iloc[0]['deg'])]):.6f}")